In [1]:
import os
import dspy

15:20:47 - LiteLLM:WARNING: get_model_cost_map.py:213 - LiteLLM: Failed to fetch remote model cost map from https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json: _ssl.c:1063: The handshake operation timed out. Falling back to local backup.


In [2]:
lm = dspy.LM(
    model="groq/openai/gpt-oss-20b",
    api_key=os.environ.get("GROQ_API_KEY"),
)

dspy.configure(lm=lm)

In [13]:
class FibonacciCheck(dspy.Signature):
    """Check the Fibonacci series and return based on user's query. 1, 1, 2, 3, 5..."""
    number: int = dspy.InputField(description="User's input number")
    position: int = dspy.OutputField(
        description="Position of this number in the Fib Seq")


math_analyser = dspy.ProgramOfThought(FibonacciCheck)

response = math_analyser(number=34)
print(response)

Prediction(
    reasoning='The function `fibonacci_position` starts from the first two Fibonacci numbers (1 and 1) and iteratively generates subsequent terms until it reaches the input number. It keeps a 1‑based index `idx`, incrementing it each time a new Fibonacci number is produced. For the input 34, the generated sequence is:\n\n1 (idx\u202f=\u202f1), 1 (idx\u202f=\u202f2), 2 (3), 3 (4), 5 (5), 8 (6), 13 (7), 21 (8), **34 (9)**\n\nWhen the loop encounters the value 34, `idx` is 9, which the function returns. The program then prints this value, resulting in the output `"9\\n"`. Hence the position of 34 in the Fibonacci sequence is 9.',
    position=9
)


In [41]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Galgotias dog")

'1 week ago - A professor claimed that a robotic dog made by Chinese firm Unitree had been made by the university. 1 week ago - It is not merely a machine on display; it is a classroom in motion. Our students are experimenting with it, testing its limits and, in the process, expanding their own knowledge. Let us be clear: Galgotias has not built this robodog, nor have we ever claimed to,” the university said in a statement posted on X. When did Galgotias University get recognition? Who opposed it at that time? Read the answers to all these questions · In the video, University Professor Neha Singh had said that this robotic dog\'s name is \'Orion\' . It has been developed by Galgotias University\'s ‘Center of Excellence’ Published 4 days ago 1 week ago - In the first statement, Galgotias University said it had not built or claimed to have built the robotic dog and emphasised its focus on student learning through exposure to advanced global technologies."Let us be clear - Galgotias has n

In [42]:
def search_web(query: str) -> str:
    """
    Search content from the Web.

    Args:
        query: str
    
    Output:
        Search response in type str
    """
    return search.invoke(query)

In [50]:
class ReActSignature(dspy.Signature):
    question = dspy.InputField(description="Users question (may need web search)")
    response = dspy.OutputField(description="Plain text - NO MARKDOWN, short concise, 30-50 words")

In [51]:
react_module = dspy.ReAct(signature=ReActSignature, tools=[search_web], max_iters=2)

prompt = "galgotia ai dog"

react_res = react_module(question=prompt)
react_res.response

'Galgotia AI Dog is a Unitree Go2, a Chinese four‑legged robotic dog showcased by Galgotia University at an AI summit. It demonstrates advanced mobility, autonomous navigation, and programmable behaviors, highlighting the university’s focus on robotics and AI education.'

In [52]:
text = "Write a poem in Hindi"

cot_completions = []
cot_emotion = dspy.ChainOfThought(signature='input -> hindi_poem')

for i in range(3):
    temp_config = dict(temperatures=0 + 0.25 * (i + 1))
    completion = cot_emotion(input=text)
    cot_completions.append(completion)

cot_completions

[Prediction(
     reasoning='I will craft a short Hindi poem that reflects on the beauty of nature and the passage of time. The poem will use simple, evocative language suitable for a general audience, and will be composed of three stanzas, each with four lines, to maintain a rhythmic flow. I will ensure that the poem stays within a concise format while expressing a reflective mood.',
     hindi_poem='सूरज की पहली किरण जब धरती पर उतरे,  \nफूलों ने खुलकर नाचना शुरू कर दिया।  \nचमकते पंखुड़ियों का यह सवेरा,  \nनयी उम्मीदों की कहानी कहता है।\n\nहवा ने पत्तों के बीच से फुसफुसाया,  \nछिपे हुए सपनों की एक धुन सुनाई।  \nसमय के सागर में तैरते हुए,  \nहम सब मिलकर एक नया गीत गाएँ।\n\nरात के तारे जब फिर से जगमगाए,  \nआकाश ने चांदनी की कहानी बुन ली।  \nख्वाबों के इस सफ़र में हम साथ चलें,  \nसुख और दुःख के साथ, जीवन के गीत गुनगुनाएँ।'
 ),
 Prediction(
     reasoning='I will craft a short Hindi poem that reflects on the beauty of nature and the passage of time. The poem will use simple, evocative la

In [60]:
multi_chain = dspy.MultiChainComparison(signature='input -> hindi_poem')

res = multi_chain(completions=cot_completions, input="Best one")
res

Prediction(
    rationale='The user’s input “Best one” implies a request for a high‑quality Hindi poem.  I interpret this as wanting a concise, evocative poem that can appeal to a general audience.  To meet that expectation, I will compose a short poem in Hindi that reflects on nature and the passage of time, using simple yet lyrical language.  The poem will consist of three stanzas of four lines each, ensuring a rhythmic flow and a balanced structure.',
    hindi_poem='सूरज की पहली किरण जब धरती पर उतरे,  \nपत्तों पर बूँदें चाँदनी की तरह झिलमिलें।  \nवो क्षण है शीतल हवा का,  \nफूलों में बसी खुशबू की गूँज।\n\nपहाड़ों की पगडंडी पर चलूँ,  \nजहाँ धुंध बुनती है रहस्य के ताने‑बाने।  \nनदियाँ कहेंगी अपनी कहानी,  \nसमय की धारा में बसी यादें।\n\nरात के सन्नाटे में तारे गुनगुनाएँ,  \nचाँदनी छाया में सपनों के पंख।  \nजीवन के हर मोड़ पर हम पाते,  \nसुख‑दुख के संग, अनंत गीतों का संग।'
)